In [1]:
import yaml
import pandas as pd
from math import ceil

from utils import *

In [2]:
generate_inventory(verbose=True)

Total Monies: 1100.45
Saving inventory data with (114, 6) shape


In [3]:
get_character_needs(['Melue', 'Zegna'])

{'Melue': {'Thistle Tea': -2}, 'Zegna': {}}

In [4]:
generate_auction_history(verbose=True)

10195 auction events
7128 auction events of interest


In [6]:
df = pd.read_parquet('intermediate/auctions.parquet')

In [12]:
df[df['timestamp']>='2020-02-01']

,auction_type,item,character,count,price,agent,timestamp,price_per
index,,,,,,,,
0,bought,Stranglekelp,Bluem,5,7311,Melmorta,2020-02-17 20:33:21,1462.2000
1,bought,Stranglekelp,Bluem,7,10227,Ariadnye,2020-02-17 20:33:20,1461.0000
2,bought,Stranglekelp,Bluem,4,5820,Dograbbit,2020-02-17 20:33:16,1455.0000
3,bought,Stranglekelp,Bluem,2,2000,Brixonguy,2020-02-17 20:33:11,1000.0000
4,bought,Stranglekelp,Bluem,1,999,Csw,2020-02-17 20:33:08,999.0000
...,...,...,...,...,...,...,...,...
10175,bought,Gromsblood,Zegna,9,27900,Alodar,2020-02-04 14:21:26,3100.0000
10176,bought,Gromsblood,Zegna,6,18598,Bnkr,2020-02-04 14:21:23,3099.6667
10180,bought,Elixir of the Mongoose,Zegna,2,101200,Litheria,2020-02-01 17:25:14,50600.0000


In [54]:
def analyse_price(df, min_count=10):
    """ Given an auctions dataframe, generate mean and std price dataframes
    """
    price_mean = df.groupby(['auction_type','item']).mean()['price_per'].unstack().T
    price_count = df.groupby(['auction_type','item']).count()['price_per'].unstack().T.fillna(0).astype(int)
    price_std = df.groupby(['auction_type','item']).std()['price_per'].unstack().T

    price_mean = price_mean[price_count>=min_count]
    price_std = price_std[price_count>=min_count]
        
    return price_mean, price_std

In [46]:
df[df['timestamp']>='2020-02-01'].shape

(5697, 8)

In [48]:
recent_price, _ = analyse_price(df[df['timestamp']>='2020-03-01'])

In [49]:
historic_price, historic_std = analyse_price(df)

In [51]:
historic_price.head()

auction_type,bought,failed,sold
item,,,
Arcane Crystal,582747.386364,NaN,NaN
Arcane Elixir,NaN,NaN,9764.837209
Arcanite Bar,NaN,770895.25,691226.594595
Blackmouth Oil,4215.625000,NaN,NaN
Blinding Powder,NaN,NaN,1581.947368


In [53]:
recent_price

auction_type,bought,failed,sold
item,,,
Arcane Crystal,620024.400000,NaN,NaN
Arcane Elixir,NaN,NaN,9404.781250
Arcanite Bar,NaN,NaN,714042.478261
Blackmouth Oil,NaN,NaN,NaN
Blindweed,3274.748739,NaN,NaN
Dream Dust,1899.616183,NaN,NaN
Dreamfoil,5334.146977,NaN,NaN
Elemental Sharpening Stone,79559.733333,NaN,NaN
Elixir of Fortitude,NaN,NaN,16186.928571


auction_type,bought,failed,sold
item,,,
Arcane Crystal,582747.386364,NaN,NaN
Arcane Elixir,NaN,NaN,9764.837209
Arcanite Bar,NaN,770895.250000,691226.594595
Blackmouth Oil,4215.625000,NaN,NaN
Blinding Powder,NaN,NaN,1581.947368
Blindweed,3409.246825,NaN,4758.407407
Dream Dust,1904.501002,NaN,NaN
Dreamfoil,5860.448206,NaN,7400.123077
Elemental Sharpening Stone,72906.666667,NaN,NaN
